# Previsão de Estoque para 28 dias

In [3]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from google.cloud import bigquery
from google.oauth2 import service_account
import logging
import matplotlib.pyplot as plt
from datetime import timedelta
from google.cloud import exceptions
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [4]:

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Caminho para o arquivo de chave da conta de serviço
service_account_path = 'tfm-sa.json'  # Substitua pelo caminho correto do seu arquivo de credenciais
credentials = service_account.Credentials.from_service_account_file(service_account_path)
project_id = 'perseverance-332400'    # Substitua pelo ID do seu projeto
dataset_id = 'TFM'                     # Substitua pelo nome do seu dataset

# Definir IDs de tabela separadas
data_table_id = 'ds_market'            # Tabela com dados originais
forecast_table_id = 'ds_market_forecast'  # Nova tabela para previsões

full_data_table_id = f'{project_id}.{dataset_id}.{data_table_id}'

# Função para configurar o cliente BigQuery usando credenciais específicas
def initialize_bigquery_client():
    return bigquery.Client(project=project_id, credentials=credentials)

In [11]:

# Função para extrair dados históricos do BigQuery
def get_historical_data_from_bigquery(query: str, client) -> pd.DataFrame:
    try:
        query_job = client.query(query)
        data = query_job.to_dataframe()
        logging.info(f"Dados extraídos do BigQuery com {data.shape[0]} linhas e {data.shape[1]} colunas.")
        logging.info(f"Colunas disponíveis nos dados extraídos: {data.columns.tolist()}")
        return data
    except exceptions.GoogleCloudError as e:
        logging.error(f"Erro ao extrair dados do BigQuery: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio para evitar falhas subsequentes

# Função para verificar estacionariedade usando o teste ADF
def check_stationarity(data: pd.Series):
    adf_test = adfuller(data)
    logging.info(f'Resultado ADF Statistic: {adf_test[0]}, Valor-p: {adf_test[1]}')
    if adf_test[1] > 0.05:
        logging.warning("A série não é estacionária.")
        return False
    else:
        logging.info("A série é estacionária.")
        return True

# Função para preparar os dados
def prepare_sales_data(raw_data: pd.DataFrame) -> pd.DataFrame:
    raw_data = raw_data.copy()
    raw_data['date'] = pd.to_datetime(raw_data['date'])
    raw_data['actual_sales'] = raw_data['actual_sales'].fillna(0)
    sales_data = raw_data.groupby('date').agg({'actual_sales': 'sum'}).sort_index()
    if sales_data.isnull().any().any():
        logging.warning("Existem datas sem dados de vendas.")
    sales_data = sales_data.asfreq('D')
    sales_data['actual_sales'] = sales_data['actual_sales'].fillna(0)
    logging.info("Dados de vendas preparados e limpos.")
    return sales_data

# Função para treinar o modelo ARIMA usando pmdarima
def train_forecast_model(data: pd.DataFrame):
    logging.info("Treinando o modelo ARIMA com pmdarima...")
    series = data['actual_sales']

    try:
        model = auto_arima(
            series,
            seasonal=False,
            stepwise=True,
            suppress_warnings=True,
            error_action='ignore',
            trace=True
        )
        order = model.order
        logging.info(f"Modelo treinado com sucesso com ordem {order}.")
        return model, order
    except Exception as e:
        logging.error(f"Erro ao ajustar o modelo ARIMA com pmdarima: {e}")
        raise

# Função para gerar previsões
def predict_sales(data: pd.DataFrame, model, steps=30, forecast_index=None) -> pd.DataFrame:
    logging.info(f"Gerando previsões para {steps} dias...")
    if model is None:
        raise ValueError("Modelo inválido. Não é possível gerar previsões.")
    forecast = model.predict(n_periods=steps)
    if forecast_index is not None:
        forecast_df = pd.DataFrame({'date': forecast_index, 'forecast_sales': forecast})
    else:
        forecast_df = pd.DataFrame({
            'date': pd.date_range(data.index[-1] + timedelta(days=1), periods=steps),
            'forecast_sales': forecast
        })
    logging.info("Previsão gerada com sucesso.")
    return forecast_df

# Função para carregar previsões no BigQuery
def store_forecast_results(data: pd.DataFrame, table_id: str, client):
    logging.info(f"Carregando previsões para a tabela {table_id} no BigQuery...")
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    job = client.load_table_from_dataframe(data, table_id, job_config=job_config)
    job.result()
    logging.info("Previsões carregadas com sucesso no BigQuery.")

# Função para plotar previsões vs. dados reais
def plot_forecast_with_actual(train_data: pd.DataFrame, test_data: pd.DataFrame, forecast: pd.DataFrame, store: str, item: str):
    plt.figure(figsize=(12, 6))
    plt.plot(train_data.index, train_data['actual_sales'], label='Dados de Treinamento')
    plt.plot(test_data.index, test_data['actual_sales'], label='Vendas Reais (Teste)')
    plt.plot(forecast['date'], forecast['forecast_sales'], label='Previsão', linestyle='--')
    plt.legend()
    plt.title(f'Previsão vs. Vendas Reais - Loja {store} Item {item}')
    plt.xlabel('Data')
    plt.ylabel('Vendas')
    plt.show()

# Pipeline completo para previsão de estoque dos itens por loja
def run_forecast_pipeline_for_top_items_per_store(query: str, forecast_table_full_id: str):
    client = initialize_bigquery_client()
    raw_data = get_historical_data_from_bigquery(query, client)

    # Verificar se as colunas necessárias existem
    required_columns = {'store', 'item', 'date', 'actual_sales'}
    if not required_columns.issubset(raw_data.columns):
        missing = required_columns - set(raw_data.columns)
        raise ValueError(f"As colunas a seguir estão faltando nos dados: {missing}")

    # Obter lista de lojas únicas
    stores = raw_data['store'].unique()
    all_forecasts = []

    for store in stores:
        logging.info(f"\nProcessando loja {store}...")
        store_data = raw_data[raw_data['store'] == store]
        total_sales_per_item = store_data.groupby('item')['actual_sales'].sum().reset_index()
        top_items = total_sales_per_item.sort_values(by='actual_sales', ascending=False).head(1)['item']  # número de itens a serem previstos

        for item in top_items:
            logging.info(f"Processando item {item} na loja {store}...")
            item_data = store_data[store_data['item'] == item]
            prepared_data = prepare_sales_data(item_data)

            if len(prepared_data) < 2:
                logging.warning(f"Dados insuficientes para item {item} na loja {store}, pulando...")
                continue

            try:
                # Dividir os dados em treinamento e teste
                train_data = prepared_data[prepared_data.index < '2015-12-01']
                test_data = prepared_data[prepared_data.index >= '2015-12-01']

                if len(train_data) < 2 or len(test_data) == 0:
                    logging.warning(f"Dados insuficientes para treinamento ou teste para item {item} na loja {store}, pulando...")
                    continue

                model, order = train_forecast_model(train_data)
                if model is None:
                    logging.error(f"Não foi possível ajustar o modelo para item {item} na loja {store}.")
                    continue

                # Prever para o período de teste
                steps = len(test_data)
                forecast = predict_sales(train_data, model, steps=steps, forecast_index=test_data.index)
                forecast['store'] = store
                forecast['item'] = item

                # Comparar previsões com vendas reais
                forecast['actual_sales'] = test_data['actual_sales'].values

                # Calcular métricas de avaliação
                rmse = mean_squared_error(forecast['actual_sales'], forecast['forecast_sales'], squared=False)
                mae = mean_absolute_error(forecast['actual_sales'], forecast['forecast_sales'])
                mape = np.mean(np.abs((forecast['actual_sales'] - forecast['forecast_sales']) / (forecast['actual_sales'] + 1e-5))) * 100  # Evitar divisão por zero

                logging.info(f"Loja {store}, Item {item} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2f}%")

                # Adicionar informações ao DataFrame de previsões
                forecast['rmse'] = rmse
                forecast['mae'] = mae
                forecast['mape'] = mape
                forecast['order'] = str(order)

                all_forecasts.append(forecast)

                # Plotar previsões vs. vendas reais
                plot_forecast_with_actual(train_data, test_data, forecast, store, item)

            except Exception as e:
                logging.error(f"Erro ao processar item {item} na loja {store}: {e}")
                continue

    if all_forecasts:
        final_forecast = pd.concat(all_forecasts, ignore_index=True)
        # Armazenar previsões na tabela de previsões
        store_forecast_results(final_forecast, forecast_table_full_id, client)
    else:
        logging.warning("Nenhuma previsão foi gerada.")

# Definir a consulta SQL para carregar os dados de 2015
query = f"""
SELECT *
FROM `{full_data_table_id}`
WHERE EXTRACT(YEAR FROM date) = 2015
"""

# Executar o pipeline, especificando a tabela de previsões
run_forecast_pipeline_for_top_items_per_store(
    query,
    f'{project_id}.{dataset_id}.{forecast_table_id}'
)

2024-10-26 20:36:18,895 - INFO - Dados extraídos do BigQuery com 3100 linhas e 9 colunas.
2024-10-26 20:36:18,897 - INFO - Colunas disponíveis nos dados extraídos: ['date', 'forecast_sales', 'store', 'item', 'actual_sales', 'rmse', 'mae', 'mape', 'order']
2024-10-26 20:36:18,903 - INFO - 
Processando loja Harlem...
2024-10-26 20:36:18,914 - INFO - Processando item SUPERMARKET_3_586 na loja Harlem...
2024-10-26 20:36:18,926 - INFO - Dados de vendas preparados e limpos.
2024-10-26 20:36:18,928 - WARNING - Dados insuficientes para treinamento ou teste para item SUPERMARKET_3_586 na loja Harlem, pulando...
2024-10-26 20:36:18,930 - INFO - 
Processando loja South_End...
2024-10-26 20:36:18,935 - INFO - Processando item SUPERMARKET_3_586 na loja South_End...
2024-10-26 20:36:18,940 - INFO - Dados de vendas preparados e limpos.
2024-10-26 20:36:18,941 - WARNING - Dados insuficientes para treinamento ou teste para item SUPERMARKET_3_586 na loja South_End, pulando...
2024-10-26 20:36:18,942 - I